# Tag Central
Here we will manipulate data from the data folder(US Election tags and Twitch plays pokemon tags).


In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AffinityPropagation
import numpy as np
import pandas as pd
import distance

In [2]:
twitch_data = pd.read_csv("data/Twitch Plays Pokemon Identifiers.csv", low_memory=False, encoding = "ISO-8859-1")
election_data = pd.read_csv("data/US Election Identifiers.csv", low_memory=False, encoding = "ISO-8859-1")
DATA = twitch_data

Twitch data and election data are loaded using panda.
Each dataset has two columns **Identifier** and **Subject**
The __tokenize_tags__ function below takes each row of tags, splits them up into arrays and puts them all together into a tags array.

In [45]:
def tokenize_tags(data):
    tags = data['Subject']
    all_tags = []
    for tag_string in tags:
        tag_string = str(tag_string)
        all_tags.extend(tag_string.split(","))
    all_tags = np.asarray(all_tags)
    return all_tags

TAGS = tokenize_tags(DATA)
print(list(TAGS[0:50]))
print("Total number of tags", len(TAGS))

['twitch', 'irc', 'twitch plays pokÃ©mon', 'tpp', 'pokÃ©mon', 'pokemon', 'pokemon red', 'pokÃ©mon red', 'red', 'nan', 'twitch', 'twitch plays pokemon', 'pokemon', 'tpp', 'pokemon', 'screenshot', 'twitchplayspokemon', 'democracy', 'emulator', 'twitch', 'twitch', 'pokemon', 'save file', 'emulator', 'twitch', 'pokemon', 'save file', 'emulator', 'twitch plays pokemon', 'twitch', 'pokemon', 'save file', 'emulator', 'twitchplayspokemon', 'twitch', 'pokemon', 'save file', 'emulator', 'twitch', 'pokemon', 'save file', 'emulator', 'twitch', 'pokemon', 'save file', 'emulator', 'pokemon', 'twitch', 'student', 'radio']
Total number of tags 302


In [91]:
#bla = 'ABC'
#print(bla.lower())
#print(bla[0:len(bla)].lower())
#blee = 'ABCDE'
#bla.lower() == blee[0:len(bla)].lower()

dic = {'key1':['value1'], 'key2':'value2'}

#if 'key1' in dic:
#     dic['key1'].append('value2')        
#dic['key2'] = [(dic['key2'])]


print(dic)
print(dic['key1'])

{'key1': ['value1'], 'key2': 'value2'}
['value1']


In [88]:
def autocomplete (inString, tagList, numRec):
    """Suggest an autocompletion of a partially entered tag.
    Input: string to be autocompleted, list of tags, number of recommendationds desired.
    Output: list of recommendations in order of popularity, with the most popular version of the tag (pre-cleanup) reocommended."""
    #First, add indices of all tags whose first letters match the user input string to a list.
    matchList = [] #Contains indices of matches to instring
    for index, tag in enumerate(tagList):
        if inString.lower() == tag[0:len(inString)].lower():
            matchList.append(index)
    print("Match List:", matchList)
    
    #Second, group the items in matchList into sets that match after cleanup.
    matchDic = {} #Dictionary w/ term:matching indices
    for matchIndex in matchList:
        if tagList[matchIndex].lower() not in matchDic: #If cleaned up word is not yet in the dictionary, make it a new key within a list and add its index as a value
            matchDic[tagList[matchIndex].lower()] = [matchIndex]
        else: #If already in dictionary, add its index to that key's value
            matchDic[tagList[matchIndex].lower()].append(matchIndex)
    print('\nMatch Dictionary:', matchDic)
    
    #Third, count the number of entries in each cleaned up dictionary entry and add it to the dictionary
    for key in matchDic:
        matchDic[key] = len(matchDic[key]), matchDic[key]
    print('\nMatch Dictionary w/ Length:', matchDic, '\n')
    
    #Fourth, identify the most popular terms (an amount equal to numRec) and add their dictionaries to a list in popularity order.
    resultsList = [] 
    for key in matchDic:
        resultsList.append({key: matchDic[key]})
    #resultsList.sort(key=lambda matchDic: matchDic[key], reverse=True)
    for index, result in enumerate(resultsList):
        for key in result:
            print("index:", index)
            print("resultsList[index]:", resultsList[index])
            print("resultsList[index][key][0]:", resultsList[index][key][0])
            if resultsList[index][key][0] < resultsList[index + 1][key][0]: #Second 'key' isn't the key referenced in for loop
                temp = resultsList[index]
                resultsList[index + 1] = resultsList[index]
                resultsList[index] = temp

    resultsList = resultsList[0:numRec - 1]

    print('\nResults List:', resultsList)
    
    
    recList = [] #Contains recommended tags, in most popular capitalization.
    
    return recList
    
autocomplete ('pok', TAGS, 10)

Match List: [4, 5, 6, 7, 12, 14, 21, 25, 30, 35, 39, 43, 46, 58, 65, 69, 75, 80, 86, 88, 92, 97, 104, 110, 115, 121, 128, 135, 143, 153, 164, 171, 178, 184, 190, 195, 196, 197, 198, 199, 200, 205, 206, 207, 208, 209, 210, 211, 218, 219, 220, 221, 222, 223, 224, 225, 226, 244, 259, 261, 294]

Match Dictionary: {'pokã©mon': [4], 'pokemon': [5, 12, 14, 21, 25, 30, 35, 39, 43, 46, 58, 65, 69, 75, 80, 86, 88, 92, 97, 104, 110, 115, 121, 128, 135, 143, 153, 164, 206, 226, 244, 259, 261, 294], 'pokemon red': [6], 'pokã©mon red': [7], 'pokã©mon trading card game online': [171, 178, 184, 190], 'pokemon memes': [195], 'pokemon memes 2014': [196], 'pokemon memes funny': [197], 'pokemon memes 1': [198], 'pokemon memes 5': [199], 'pokemon memes 6': [200], 'pokemon shaming is now a thing': [205], 'pokemon omega ruby and alpha sapphire': [207], 'pokemon x and y': [208], 'pokã©mon (brand)': [209], 'pokã©mon (tv program)': [210], 'pokã©mon (video game series)': [211], 'pokã©mon go - mobile game': [218]

KeyError: 'pokã©mon'

__________________________________________________________________________________
__________________________________________________________________________________

## Clustering
The levenshtein distance calculates how similar words are to each other based on how many steps it would take to convert one
word to the other using deletion, and creation of new characters. This method is not as effective as cosine similarity. It's also very slow.

In [4]:
#lev_similarity = -1 * np.array([[distance.levenshtein(t1.lower(),t2.lower()) for t1 in TAGS] for t2 in TAGS])
#lev_similarity

Here we use TFIDF vectorization to convert words to numbers and use cosine similarity function to determine how similar words are to each other. It is quick but not perfect as seen in the clustering algorithms below.

In [5]:
tfidf_vectorizer=TfidfVectorizer()
tfidf_matrix=tfidf_vectorizer.fit_transform(TAGS)
cs_similarity = np.array([cosine_similarity(tfidf_matrix[i:i+1],tfidf_matrix).flatten() for i in range(len(TAGS))])
cs_similarity

array([[ 1.        ,  0.        ,  0.38920701, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.38920701,  0.        ,  1.        , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  1.        ,
         0.        ,  1.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  1.        ,
         0.        ,  1.        ]])

This function uses Affinity Propagation to cluster words that are most similar together. It outputs a dictionary that looks like this :

{ 'Most popular tag called exemplar' : [ tags similar to exemplar comma separated ] }

We can explore other clustering algorithms as well.

In [14]:
def cluster(data, tags):
    affprop = AffinityPropagation(affinity="precomputed", damping=0.5)
    affprop.fit(data)
    clustered_tags = {}
    for cluster_id in np.unique(affprop.labels_):
        exemplar = tags[affprop.cluster_centers_indices_[cluster_id]].lower()
        if exemplar in list(clustered_tags.keys()):
            arr = clustered_tags[exemplar]
        else:
            arr = []
        cluster = np.unique(tags[np.nonzero(affprop.labels_==cluster_id)])
        arr.extend(cluster.tolist())
        clustered_tags[exemplar] =  list(set(arr))
        cluster_str = ", ".join(cluster)
    print("No, of labels", len(clustered_tags.keys()))
    return clustered_tags

Cosine similarity works a little better than levenshtein distaance in predicting more similar words.
We can explore other ways of doing this.

In [15]:
#clustered_tags  = cluster(lev_similarity)
clustered_tags = cluster(cs_similarity, TAGS)
clustered_tags

No, of labels 42


{'3 hit combo podcast': ['ips', '3 hit combo podcast', '3 Hit Combo Podcast'],
 'anime': ['dublin', 'anime'],
 'battle': ['pokemon go battle', 'battle'],
 'belfield': ['belfield'],
 'chroma': ['fm', 'Chroma'],
 'comedy': ['comedy', 'Comedy', 'comments'],
 'democracy': ['democracy'],
 'emulator': ['ireland',
  'traveling woes',
  'People & Blogs',
  'regional observations',
  'oqt',
  'dec3199',
  'bort',
  'viet crystal',
  'nerd',
  'revo',
  'Blizzard',
  'emulator',
  'nan',
  'J#SM',
  'xd minglee'],
 'funny': ['funny pokemon pictures',
  'funny picture',
  'Funny',
  'funny',
  'funny Pokemon memes pictures',
  'Pokemon memes funny'],
 'gametrailers': ['GameTrailers'],
 'gaming': ['gaming', 'Gaming'],
 'geek': ['geek'],
 'jolly swag men': ['#swag', 'Jolly Swag Men', 'Jolly #Swag Men'],
 'live stream': ['live', 'live stream'],
 'nerds': ['nerds', 'Fallout 4'],
 'news': ['News', 'news'],
 'nintendo': ['nintendo'],
 'pinball': ['Pinball'],
 'podcast': ['podcast', 'Podcast'],
 'pokemo

## Popularity Index
Here we will prepare a popularity index dictionary that will map each (exemplar) tag to a count. The count indicates how many documents have that tag. 
This will be used in autocompletion

In [16]:
def popularity(clustered_tags, all_tags):
    popularity_index = {}
    for exemplar in clustered_tags:
        count = 0
        arr = clustered_tags[exemplar]
        arr.append(exemplar)
        for tag in all_tags:
            if tag and tag in arr:
                count +=1
        popularity_index[exemplar] = count
    return popularity_index
                
popularity_index = popularity(clustered_tags, TAGS)

## Inverted Index
Here we are preparing an inverted index of our tags and identifiers

First, we convert the dataframe to a dictionary. The key is the identifier and the  the value is a string of comma separated tags.
The **make_inverted_index** function that converts this dictionary into a dictionary where the key is the (exemplar) tag and the value is a list of documents where is occurs. The documents are labelled by their position. e.g. 0,1,2,3. This is much easier to work with than their longer values e.g. live_user_twitchplayspokemon_1407024801

In [18]:
DATA = DATA.set_index('Identifier').T.to_dict('list')

AttributeError: 'dict' object has no attribute 'set_index'

In [19]:
def make_inverted_index(data, clustered_tags):
    inverted_index = {}
    for i, doc in enumerate(data):
        doc_tags = str(data[doc][0]).split(",")
        for j, exemplar in enumerate(clustered_tags):
            arr = clustered_tags[exemplar]
            arr.append(exemplar)
            for tag in doc_tags:
                if tag in arr:
                    if inverted_index.get(exemplar, None):
                        inverted_index[exemplar].append(i)  
                    else:
                        inverted_index[exemplar] = [i]
    return inverted_index
                     
inverted_index = make_inverted_index(DATA, clustered_tags)
inverted_index

{'3 hit combo podcast': [28, 42, 44],
 'anime': [11, 40, 41],
 'battle': [22, 38],
 'belfield': [11],
 'chroma': [11, 46],
 'comedy': [22, 42, 44],
 'democracy': [3, 30],
 'emulator': [1,
  3,
  4,
  5,
  7,
  8,
  9,
  10,
  11,
  11,
  12,
  14,
  15,
  16,
  17,
  19,
  20,
  21,
  22,
  23,
  24,
  24,
  25,
  26,
  26,
  27,
  28,
  28,
  29,
  30,
  30,
  30,
  30,
  38,
  39,
  39,
  42,
  48],
 'funny': [37, 37, 37, 37, 42, 44],
 'gametrailers': [13],
 'gaming': [13, 37, 42, 44, 46],
 'geek': [11, 45],
 'jolly swag men': [48, 48, 48],
 'live stream': [22, 32, 34, 35, 36],
 'nerds': [40, 41, 46],
 'news': [42, 44, 45],
 'nintendo': [22],
 'pinball': [43, 47],
 'podcast': [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
 'pokemon': [0,
  2,
  3,
  4,
  5,
  7,
  8,
  9,
  10,
  11,
  12,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  30,
  31,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  38,
  38,
  38,
  38,
  38,
  38,
  38,
  38

In [20]:
def get_similar_tags(tag, clustered_tags, inverted_index):
    array_of_exemplars = [key for key in clustered_tags]
    if tag.lower() not in array_of_exemplars:
        return []
    index  = array_of_exemplars.index(tag.lower())
    array_of_docs = [','.join(str(doc) for doc in inverted_index[exemplar]) for exemplar in array_of_exemplars]
    tfidf = TfidfVectorizer().fit_transform(array_of_docs)
    cosine_similarities = cosine_similarity(tfidf[index:index+1], tfidf).flatten()
    most_similar_tags = cosine_similarities.argsort()[:-5:-1]
    similar_tags = [array_of_exemplars[i] for i in most_similar_tags]
    return similar_tags[1:]

In [21]:
get_similar_tags("pokemon", clustered_tags, inverted_index)

['pokã©mon', 'youtube', 'funny']

Using the andSearch from homework 4
    